In [1]:
import os
from dotenv import load_dotenv
import mlflow 
from mlflow import MlflowClient

load_dotenv()

True

In [2]:
# conectar con mlflow y minio (por http)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://127.0.0.1:9000"
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('ACCESS_KEY')

# Model operations

In [15]:
client = MlflowClient()
model_name = "MyModel"

## Create registered model

In [3]:
# create model
class MyModel(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input):
        return model_input.apply(lambda column: column*2)

In [39]:
mlflow.set_experiment("mlflow_tracking")

with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(artifact_path="model", python_model=MyModel(), registered_model_name=model_name)
    print("RUN ID:", run.info.run_id)

Registered model 'MyModel' already exists. Creating a new version of this model...
2024/06/19 20:35:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MyModel, version 5


RUN ID: 8a6b0e9dc5a24eefaa99c676d8b3c500


Created version '5' of model 'MyModel'.


In [20]:
print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

tracking uri: http://127.0.0.1:5000
artifact uri: s3://mlflow/3/4f390466135b40e48ee9239537d957c7/artifacts


In [34]:
mlflow.end_run()

## Create model version 

In [59]:
with mlflow.start_run() as run:
    client.set_model_version_tag(name=model_name, version="1", key="tag_key", value="tag_value")

In [60]:
mlflow.end_run()

## Delete tag model version 

In [65]:
with mlflow.start_run(run_name="logging_artifacts27") as run:
    client.delete_model_version_tag(name=model_name, version="1", key="tag_key2")

In [66]:
mlflow.end_run()

## Adding description to registired model

In [64]:
with mlflow.start_run() as run:
    client.update_registered_model(name=model_name, description="This is a test model")

In [67]:
mlflow.end_run()

## Adding description to model version

In [68]:
with mlflow.start_run() as run:
    client.transition_model_version_stage(name=model_name, version=1, stage="Archived")
    client.transition_model_version_stage(name=model_name, version=2, stage="Archived")
    client.transition_model_version_stage(name=model_name, version=3, stage="Archived")
    client.transition_model_version_stage(name=model_name, version=4, stage="Staging")
    client.transition_model_version_stage(name=model_name, version=5, stage="Production")

/tmp/ipykernel_135832/1796222191.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(name=model_name, version=1, stage="Archived")
/tmp/ipykernel_135832/1796222191.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(name=model_name, version=2, stage="Archived")
/tmp/ipykernel_135832/1796222191.py:4: FutureWarning: ``mlflow.t

In [70]:
mlflow.end_run()

In [71]:
print('ok_')

ok_
